In [1]:
from preprocessing import *
from sklearn.model_selection import KFold
import argparse
from model import *
from train import test
import torch.optim as optim
import pandas as pd

from MatrixVectorizer import *
import networkx as nx
from typing import Union


In [2]:
# load csvs as numpy
lr_data_path = '../data/lr_train.csv'
hr_data_path = '../data/hr_train.csv'

lr_train_data = pd.read_csv(lr_data_path, delimiter=',').to_numpy()
hr_train_data = pd.read_csv(hr_data_path, delimiter=',').to_numpy()
lr_train_data[lr_train_data < 0] = 0
np.nan_to_num(lr_train_data, copy=False)

hr_train_data[hr_train_data < 0] = 0
np.nan_to_num(hr_train_data, copy=False)

# map the anti-vectorize function to each row of the lr_train_data

lr_train_data_vectorized = np.array([MatrixVectorizer.anti_vectorize(row, 160) for row in lr_train_data])
hr_train_data_vectorized = np.array([MatrixVectorizer.anti_vectorize(row, 268) for row in hr_train_data])

num_samples = hr_train_data_vectorized.shape[0]
num_samples_list = range(num_samples)
sample_to_index = dict(zip(num_samples_list, hr_train_data_vectorized))

split = int(num_samples * 0.8)

In [3]:
lr_train_data_vectorized = torch.tensor(lr_train_data_vectorized, dtype=torch.float32)
hr_train_data_vectorized = torch.tensor(hr_train_data_vectorized, dtype=torch.float32)

from torch.utils.data import Dataset

class NoisyDataset(Dataset):
    def __init__(self, lr_data, hr_data, noise_level=0.01):
        """
        lr_data: Low resolution data (torch.tensor)
        hr_data: High resolution data (torch.tensor)
        noise_level: Standard deviation of Gaussian noise to be added
        """
        self.lr_data = lr_data
        self.hr_data = hr_data
        self.noise_level = noise_level

    def __len__(self):
        return len(self.lr_data)

    def __getitem__(self, idx):
        lr_sample = self.lr_data[idx]
        hr_sample = self.hr_data[idx]

        # Adding Gaussian noise
        noise = torch.randn(lr_sample.size()) * self.noise_level
        noisy_lr_sample = lr_sample + noise

        # Clipping to ensure values are between 0 and 1
        noisy_lr_sample = torch.clamp(noisy_lr_sample, 0, 1)

        return noisy_lr_sample, hr_sample

train_data = NoisyDataset(lr_train_data_vectorized, hr_train_data_vectorized, noise_level=0.5)

train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=1, shuffle=True)
sample = next(iter(train_data_loader))
print(sample[0].shape, sample[1].shape)

torch.Size([1, 160, 160]) torch.Size([1, 268, 268])


In [4]:

subjects_adj,subjects_labels = lr_train_data_vectorized[:split], hr_train_data_vectorized[:split]

held_out_subjects_adj,held_out_subjects_labels = lr_train_data_vectorized[split:], hr_train_data_vectorized[split:]

In [5]:
num_splt = 3
epochs = 50
lr = 0.00005 # try [0.0001, 0.0005, 0.00001, 0.00005]
lmbda = 17 # should be around 15-20
lamdba_topo = 1 # should be around 0.5-1.5
lr_dim = 160
hr_dim = 320
hidden_dim = 320 # try smaller and larger - [160-512]
padding = 26
dropout = 0.1 # try [0., 0.1, 0.2, 0.3]
args = argparse.Namespace()
args.epochs = epochs
args.lr = lr
args.lmbda = lmbda
args.lamdba_topo = lamdba_topo
args.lr_dim = lr_dim
args.hr_dim = hr_dim
args.hidden_dim = hidden_dim
args.padding = padding
args.p = dropout


In [6]:
cv = KFold(n_splits=3, random_state=42, shuffle=True)

In [7]:
ks = [0.9, 0.7, 0.6, 0.5]
model = GSRNet(ks, args)

In [8]:
# precompute topological measures for hr_train_data_vectorized
# precomputed_measures = precompute_topological_measures(hr_train_data_vectorized)

In [9]:
import time
criterion = nn.L1Loss()

def train(model, optimizer, subjects_adj,subjects_labels, args): 
  
  all_epochs_loss = []
  no_epochs = args.epochs

  for epoch in range(no_epochs):
    epoch_loss = []
    epoch_error = []
    epoch_topo = []

    model.train()
    for lr, hr in train_data_loader:      
      lr = lr.reshape(160, 160)
      hr = hr.reshape(268, 268)

      model_outputs,net_outs,start_gcn_outs,layer_outs = model(lr)
      model_outputs  = unpad(model_outputs, args.padding)

      padded_hr = pad_HR_adj(hr,args.padding)
      eig_val_hr, U_hr = torch.linalg.eigh(padded_hr, UPLO='U')

      loss = args.lmbda * criterion(net_outs, start_gcn_outs) + criterion(model.layer.weights,U_hr) + criterion(model_outputs, hr) 
      start_time = time.time()

      topo = args.lamdba_topo * compute_topological_MAE_loss(hr, model_outputs)

      error = criterion(model_outputs, hr)
      
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      epoch_loss.append(loss.item())
      epoch_error.append(error.item())
      epoch_topo.append(topo.item())
      
  
    model.eval()
    print("Epoch: ",epoch+1, "Loss: ", np.mean(epoch_loss), "Error: ", np.mean(epoch_error),
          "Topo: ", np.mean(epoch_topo))
    all_epochs_loss.append(np.mean(epoch_loss))

In [10]:
# # print(model)
# optimizer = optim.Adam(model.parameters(), lr=args.lr)
# # optimizer = optim.SGD(model.parameters(), lr=args.lr)

# for train_index, test_index in cv.split(subjects_adj):
#     subjects_adj_train = subjects_adj[train_index]  # Get training data 
#     subjects_adj_test = subjects_adj[test_index]   # Get testing data 
#     subjects_ground_truth_train = subjects_labels[train_index]
#     subjects_ground_truth_test = subjects_labels[test_index]

#     train(model, optimizer, subjects_adj_train, subjects_ground_truth_train, args, subjects_adj_test, subjects_ground_truth_test)
    
#     print('Held out test score:')
#     test(model, held_out_subjects_adj, held_out_subjects_labels, args)
#     print('------------------------------')

# Final Model & Kaggle Submission

In [11]:
#final train
final_model = GSRNet(ks, args)
optimizer = optim.Adam(final_model.parameters(), lr=args.lr)

train(final_model, optimizer, lr_train_data_vectorized, hr_train_data_vectorized, args)

Epoch:  1 Loss:  3.029073493923256 Error:  0.20921504310148206 Topo:  38.00805455053638
Epoch:  2 Loss:  2.7525105490655957 Error:  0.1808784367617019 Topo:  20.92716292421261
Epoch:  3 Loss:  2.611178700795431 Error:  0.17738783627213117 Topo:  20.313580587238608
Epoch:  4 Loss:  2.4653049143488537 Error:  0.1751270353972555 Topo:  19.29350514326267
Epoch:  5 Loss:  2.3236586647833177 Error:  0.17397132783592817 Topo:  19.029875098588224
Epoch:  6 Loss:  2.179130302931734 Error:  0.17284518078772607 Topo:  18.8364652587982
Epoch:  7 Loss:  2.034163193074529 Error:  0.17171803705706568 Topo:  18.905736066624076
Epoch:  8 Loss:  1.8872685196870815 Error:  0.1707447043614473 Topo:  18.73559332179452
Epoch:  9 Loss:  1.7399358185465463 Error:  0.1692367671671028 Topo:  18.52023202930382
Epoch:  10 Loss:  1.59193136663494 Error:  0.16810988773128943 Topo:  18.322864949346304
Epoch:  11 Loss:  1.443823534571482 Error:  0.16699420620581348 Topo:  18.556593894958496
Epoch:  12 Loss:  1.294654

In [12]:
#Generate submission 

# load csvs as numpy
test_lr_data_path = '../data/lr_test.csv'

# lr_test_data = np.loadtxt(test_lr_data_path, delimiter=',')
lr_test_data = pd.read_csv(test_lr_data_path, delimiter=',').to_numpy()
print(lr_test_data.shape)
lr_test_data[lr_test_data < 0] = 0
np.nan_to_num(lr_test_data, copy=False)


# map the anti-vectorize function to each row of the lr_train_data

lr_test_data_vectorized = np.array([MatrixVectorizer.anti_vectorize(row, 160) for row in lr_test_data])
print(lr_test_data_vectorized.shape)

(112, 12720)
(112, 160, 160)


In [13]:
final_model.eval()
preds = []
for lr in lr_test_data_vectorized:      
  lr = torch.from_numpy(lr).type(torch.FloatTensor)
  
  model_outputs, _, _, _ = final_model(lr)
  model_outputs  = unpad(model_outputs, args.padding)
  preds.append(MatrixVectorizer.vectorize(model_outputs.detach().numpy()))

print(len(preds), preds[0].shape)
r = np.hstack(preds)
print(r.shape)
meltedDF = r.flatten()

112 (35778,)
(4007136,)


In [14]:
n = meltedDF.shape[0]
df = pd.DataFrame({'ID': np.arange(1, n+1),
                   'Predicted': meltedDF})
df.to_csv('submission.csv', index=False)